In [1]:
import os
import random
import gym
import numpy as np

from collections import deque

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
env = gym.make('CartPole-v0')

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
state_size, action_size

(4, 2)

In [4]:
batch_size = 32
n_episodes = 1000

output_dir = '/home/mikey/Cartpole_dqn'

# if not os.path.exists(output_dir):
#     os.makedirs('Cartpole_dqn')

In [52]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        
        self.memory = deque(maxlen = 2000)
        
        self.gamma = 0.95
        
        self.epsilon = 1.0
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        
        self.learning_rate = 0.001
        
        self.model = self._build_model()
    
    def _build_model(self):
        model = Sequential([
            Dense(24, input_dim = state_size, activation = 'relu'),
            Dense(24, activation = 'relu'),
            Dense(self.action_size, activation = 'linear')
        ])
        
        model.compile(loss = 'mse', optimizer = Adam(learning_rate = self.learning_rate))
        
        return model
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values)
    
    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        
        for state, action, reward, next_state, done in minibatch:
            target = self.model.predict(state)
            
            if done:
                target[0][action] = reward
            else:
                Q_future  = self.model.predict(next_state)[0]
                target[0][action] = reward + self.gamma * np.amax(Q_future)
            self.model.fit(state, target, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
            
    def load(self, name):
        self.model.load_weights(name)
    
    def save(self, name):
        self.model.save_weights(name)
        
            
        
    

In [53]:
agent = DQNAgent(state_size, action_size)

In [ ]:
import time


done =False

for e in range(n_episodes):
    
    state = env.reset()
    state = np.reshape(state, [1,state_size])
    
    for time in range(200):
        env.render()       
        
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        env.render()
        
        reward = reward if not done else -10
        
        next_state = np.reshape(next_state, [1, state_size])
        
        agent.remember(state, action, reward, next_state, done)
        
        state = next_state
        
#         if done:
#             print("episodes {} / {}, score : {}, e {} ".format(n_episodes, time, agent.epsilon))
#             break
        
        if len(agent.memory) > batch_size:
            agent.replay(batch_size)
            
#         if e % 50 == 0:
#             agent.save(output_dir+"weights"+"{:04d}".format(e)+".hdf5")